In [1]:
!pip install opendatasets datasets accelerate --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 

100%|██████████| 400k/400k [00:00<00:00, 60.6MB/s]

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
data = pd.read_csv('/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv')[['question', 'answer']]
data.head()

,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


In [4]:
# Normalizing and cleaning the data to remove punctuation and capitalization and special characters
def clean_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z8-9]+', ' ', text)

  return text

In [5]:
data['question'] = data['question'].apply(clean_text)
data['answer'] = data['answer'].apply(clean_text)

In [6]:
data.head()

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


In [7]:
# Split the data

train = data.sample(frac=0.7)
test = data.drop(train.index)


val = test.sample(frac=0.5)
test = test.drop(val.index)

print('Training Shape:', train.shape)
print('Validation Shape:', val.shape)
print('Testing Shape:', test.shape)

Training Shape: (4976, 2)
Validation Shape: (1066, 2)
Testing Shape: (1066, 2)


In [8]:
original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype = torch.float32)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
# Model and tokenizer
def tokenize_function(example):
  start_prompt = "According to the following question:\n\n"
  end_prompt = "\nAnswer:\n\n"

  prompt = [start_prompt + question + end_prompt for question in example['question']]

  example["input_ids"] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors = 'pt').input_ids
  example["labels"] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors = 'pt').input_ids

  return example

In [10]:
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map(tokenize_function, batched = True)
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

In [11]:
val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched = True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched = True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [12]:
EPOCHS = 5
LR = 1e-3
BATCH_SIZE = 2

training_path = "./training_nvidia_chatbot"

training_args = TrainingArguments(
    output_dir = training_path,
    save_total_limit = 2,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCHS,
    learning_rate = LR,
    evaluation_strategy = "epoch",
)

trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets
)

trainer.train()

model_path = './nvidia_chatbot_final_model'

trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.182000,0.159619
2,0.134300,0.142804
3,0.092600,0.138898
4,0.061800,0.146118
5,0.037000,0.162011


('./nvidia_chatbot_final_model/tokenizer_config.json',
 './nvidia_chatbot_final_model/special_tokens_map.json',
 './nvidia_chatbot_final_model/spiece.model',
 './nvidia_chatbot_final_model/added_tokens.json',
 './nvidia_chatbot_final_model/tokenizer.json')

In [13]:
eval_results = trainer.evaluate(eval_dataset = test_tokenized_datasets)

In [14]:
print(eval_results)

{'eval_loss': 0.17181049287319183, 'eval_runtime': 118.4041, 'eval_samples_per_second': 9.003, 'eval_steps_per_second': 4.502, 'epoch': 5.0}


In [16]:
# Testing and evaluating model
test_text = 'what is cuda nsight?'

start_prompt = "According to the following question:\n\n"
end_prompt = "\nAnswer:\n\n"

full_prompt = start_prompt + test_text + end_prompt

print(full_prompt)

According to the following question:

what is cuda nsight?
Answer:




In [19]:
from transformers import GenerationConfig

trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = trained_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output, skip_special_tokens = True)

print(final_output)

cuda nsight is a preview release that provides csight for gpu kernels nsight compute provides a preview of cuda nsight compute for gpu kernels 


In [21]:
from transformers import GenerationConfig

original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype = torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = original_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output, skip_special_tokens = True)

print(final_output)

a snaretictrachetebedete etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement etablissement


In [23]:
test_data[53]

{'question': 'what are the downsides of using malloc like abstractions in cuda applications before cuda ',
 'answer': 'before cuda using malloc like abstractions in cuda applications had limitations such as limited options for memory management and inefficient dynamic data structure creation ',
 '__index_level_0__': 300}

In [24]:
test_text = 'what are the downsides of using malloc like abstractions in cuda applications before cuda '

start_prompt = "According to the following question:\n\n"
end_prompt = "\nAnswer:\n\n"

full_prompt = start_prompt + test_text + end_prompt

print(full_prompt)

According to the following question:

what are the downsides of using malloc like abstractions in cuda applications before cuda 
Answer:




In [25]:
from transformers import GenerationConfig

trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = trained_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output, skip_special_tokens = True)

print(final_output)

before cuda malloc like abstractions introduced in c standard cuda provided a single abstraction for malloc like arrays requiring the use of malloc like abstractions for these operations led to a lack of performance and a lack of abstraction for explicitly mapped arrays 


In [26]:
from transformers import GenerationConfig

original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype = torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = original_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output, skip_special_tokens = True)

print(final_output)

a large number of refactoringstystystystystystystystystystystystystystystystystystystystystystystystystystystystystystystystystystystys
